# Getting confident with Spark distributed file storage

Before we start Spark and the analysis of big data, we will look to 3 databricks useful concepts. The filesystem, the display method and the built in SQL commands. 

The concept of big data is tightly coupled with concepts like datalake and the hadoop ecosystem.  
Frequently in tutorials, spark will use hadoop as the underlying platform (before diving in spark we will look into the distributed file system backing it up).    

[What are the differences to hdfs?](https://stackoverflow.com/questions/55386728/what-are-the-main-differences-between-hdfs-and-databricks-dbfs)  
[Some _biased_ opinions on the advantages of dbfs over hdfs](https://databricks.com/blog/2017/05/31/top-5-reasons-for-choosing-s3-over-hdfs.html)

What is [Amazon’s S3, Microsoft’s Azure Blob Storage, Google’s Cloud Storage, insert your favourite cloud provider name here]?  
They are [object storage systems](https://en.wikipedia.org/wiki/Object_storage) summarized as a filesystems over the network. 

Databricks uses as a default distributed storage the [dbfs](https://docs.databricks.com/data/databricks-file-system.html), let's take some time to understand how to operate with it.

## Databricks File System (DBFS)

Here we will see some examples of how to interact with the dbfs.

In [0]:
dbutils.fs.help()

Let's use the **put** method to try and add files to the file storage

In [0]:
dbutils.fs.put("dataframe_sample.csv", """id|end_date|start_date|location
1|2015-10-14 00:00:00|2015-09-14 00:00:00|CA-SF
2|2015-10-15 01:00:20|2015-08-14 00:00:00|CA-SD
3|2015-10-16 02:30:00|2015-01-14 00:00:00|NY-NY
4|2015-10-17 03:00:20|2015-02-14 00:00:00|NY-NY
5|2015-10-18 04:30:00|2014-04-14 00:00:00|CA-SD
""", True)

Confirm the file is really there

In [0]:
dbutils.fs.ls("/")

Can be also expressed with

In [0]:
%fs ls

And to confirm the contents of the file

*/ **Tip:** In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path" translates into "dbutils.fs.head("/file/path", maxBytes = 10000)"*

In [0]:
%fs head dataframe_sample.csv

Using python to read the contents of the file.  
Next week we will look in detail at what these commands are doing. Note the ``collect`` command at the end of the next cell: Spark will avoid performing data work until an action triggers the computation chain (instructions will be cached, not datasets). The reasoning for this can be summarized as follows:
- Storing instructions in memory takes much less space than storing intermediate data results.
- By having the full list of tasks to be performed available, the master optimizes work between executors much more efficiently
- The user can iteratively build their chain of transformation, one at the time,and when ready, launch the computation

In [0]:
%python 

df = spark.read.format("csv") \
          .option("header", "true") \
          .option("inferSchema", "true") \
          .option("delimiter", "|") \
          .load("dbfs:/dataframe_sample.csv") \
          .collect()

display(df)

Bellow is a detailed description of the diference between the distributed filesystem and the local one (we will get in details later in the presentation)

In the traditional approach, all the data was stored in a single central database. With the rise of big data, a single database was not enough for storage. The solution was to use a distributed approach to store the massive amount of data. Data was divided and distributed amongst many individual databases. HDFS is used to store big data in a distributed way. Here, data is broken down into smaller chunks and stored in various machines. Not only this, it also makes copies of this data and uses these copies if one machine fails.


<img src ='https://docs.microsoft.com/en-us/azure/databricks/_static/images/data-import/dbfs-and-local-file-paths.png'>

**One important distinction is that data on the dbfs will be persisted across clusters. Meanwhile local data will be erased.**

_https://docs.databricks.com/data/databricks-file-system.html_

_Note: Althogh these is the official documentation from databricks it seems the mapping between the dbfs to the local filesystem at /dbfs is not working as expected in the community edition_

In [0]:
%sh
ls /dbfs

#### File upload using the Web UI

File upload to the dbfs is also possible with the notebook's Upload Data command

<img src =https://docs.databricks.com/_images/upload-data-menu.png>

_https://docs.databricks.com/data/databricks-file-system.html?#upload-data-to-dbfs-from-a-notebook_

## The display command

Let's get the data from last class to get a more featurefull dataset to visualize.

In [0]:
%sh
wget -O tuberculosis.csv --timeout=15 https://public.tableau.com/app/sample-data/TB_Burden_Country.csvv

Where in the file system are we?

In [1]:
%%sh
ls; pwd

Couldn't find program: 'sh'


Ok so our file in in /databricks/driver.  
Then we can import it to spark with the same command as before

In [0]:
tuberculosis_df = spark.read.format("csv") \
          .option("header", "true") \
          .option("inferSchema", "true") \
          .load("file:/databricks/driver/tuberculosis.csv")


And finally test the display command

In [0]:
display(tuberculosis_df)

By default it will display the data in table format, we need to click on the small icon to change it to graph view.


_Note: here we see that it will only load 1000 rows and only upon request it will generate visualizations with the full dataset_

---

We can then select the plot options icon and play with the data

---

By selecting x and y variables, grouping variables and visualiazation types  

---
After our round of exploration we can click apply and spark will execute the current visualization in the entire dataset.
<img src ='https://imsclasses.blob.core.windows.net/images/all_dataset_confirmation.png'>

_https://docs.databricks.com/notebooks/visualizations/index.html_

## SQL in databricks

Working with files might not always be the most intuitive way of analysing data.  
In the case of hdfs, tools like hive have been developed to provide a SQL interface to content stored in files.  
In databricks, this is hidden from the end user and SQL becomes almost native to the platform.  
Let's look at some examples.

In [0]:
# SQL code here

Here we just imported the file we inserted into the dbfs in the beggining of the this notebook to SQL

In [0]:
# SQL code here

Now let's look at the more complex data

In [0]:
# Code here

Above we see another way to load to SQL. In this case using python.  
Let's inspect the data?

In [0]:
# SQL code here

Confirm aggregations work in the same way

In [0]:
# SQL code here

In [0]:
# SQL code here

Another interesting way to see this data is to visualize it in a map with the map visualization.  
But for that we need country codes to plot them on a map so let's download it.

In [0]:
%sh 
wget -O /iso_country_codes.csv --timeout=15 "https://gist.githubusercontent.com/radcliff/f09c0f88344a7fcef373/raw/2753c482ad091c54b1822288ad2e4811c021d8ec/wikipedia-iso-country-codes.csv"

Upload it to the distributed file system

In [0]:
%fs ls

In [0]:
dbutils.fs.cp("file:/iso_country_codes.csv", "dbfs:/iso_country_codes.csv")

And create another sql table

In [0]:
# SQL code here

Let's confirm the results

In [0]:
# SQL code here

Great! Now we need to merge the 2 datasets

In [0]:
# SQL code here

Ok two problems here, the scale is very close that it gets hard to see. And some missing datapoints.  
For the scale problem we can try some different scales such as the log. Let's try to understand why do we have missing datapoints  

Which countries are in our aggregate but not on the country codes dataset?

In [0]:
# SQL code here

And now do the oposite. Which country code names are not on our dataset

In [0]:
# SQL code here

After this quick analysis we can understand that the names in one dataset do not exactly match the ones in the other. One example is Bolivia (Plurinational State of) vs. Bolivia, Plurinational State of
We will stop here. But can you think about some ideas on how to fix this problem?

## Some more examples

https://docs.databricks.com/getting-started/spark/dataframes.html

In [0]:
import seaborn as sns
sns.set(style="white")

df = sns.load_dataset("iris")
g = sns.PairGrid(df, diag_sharey=False)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3)

g.map_upper(sns.regplot)

display(g.fig)

### Example: Population versus Price

In [0]:
data = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header="true", inferSchema="true") 
data.take(10)

In [0]:
%python
display(data)

In [0]:
# Register table so it is accessible via SQL Context
data.createOrReplaceTempView("data_geo")

#### Hover over the state for 2015 Median Home Prices

In [0]:
# SQL code here

### Top 10 Cities by 2015 Median Sales Price

In [0]:
# SQL code here

### 2014 Population Estimates in Washington State

In [0]:
# SQL code here

### 2015 Median Sales Price Box Plot

Box plot shows means + variation of prices.

In [0]:
# SQL code here

### 2015 Median Sales Price by State Histogram

In [0]:
# SQL code here

### 2015 Median Sales Price by State Quantile Plot >= $ 300,000

Quantile plots help describe distributions (in this case, the distribution of sales prices across cities) and highlight aspects such as skewed distributions.

In [0]:
# SQL code here

### Cities with 2015 Median Sales Price >= $ 300,000

In [0]:
# SQL code here

### 2015 Median Sales Price Q-Q Plot

Q-Q plots provide yet another view of distributions.  See [Wikipedia on Q-Q Plots](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot) for more background.

In [0]:
# SQL code here